In [360]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import bs4 as bs
import pandas as pd

In [361]:
service = Service(ChromeDriverManager().install())


In [362]:
marca = ['xpro nutrition']
ncl = ['']

In [363]:
# basica radical 


browser = webdriver.Chrome(service=service)
browser.get("https://busca.inpi.gov.br")
time.sleep(2)
browser.find_element(By.XPATH,'//*[@id="principal"]/form/table/tbody/tr[6]/td/font/b/font/a').click();
time.sleep(2)



browser.get('https://busca.inpi.gov.br/pePI/servlet/LoginController?action=login')

# segue a trilha para a pagina de pesuisa e seleciona basica por radical
browser.find_element(By.CSS_SELECTOR, 'area[href="/pePI/jsp/marcas/Pesquisa_num_processo.jsp"]').click();
time.sleep(1)
browser.find_element(By.XPATH,'//*[@id="principal"]/table[2]/tbody/tr[1]/td/font/a[1]').click();
time.sleep(1)
browser.find_element(By.XPATH,'//*[@id="principal"]/table[2]/tbody/tr[4]/td[2]/input[2]').click();

# insere a marca e ncl na caixa de pesquisa
browser.find_element(By.XPATH,'//*[@id="principal"]/table[2]/tbody/tr[6]/td[2]/font/input').send_keys(marca);
time.sleep(1)
browser.find_element(By.XPATH,'//*[@id="principal"]/table[2]/tbody/tr[8]/td[2]/font[1]/input').send_keys(ncl);
time.sleep(1)

# seleciona o numero maximo de reistros por tela (100)
browser.find_element(By.XPATH,'//*[@id="principal"]/table[2]/tbody/tr[10]/td[2]/font/select/option[1]').click()
time.sleep(1)
#pesquisa
browser.find_element(By.XPATH,'//*[@id="principal"]/table[2]/tbody/tr[11]/td/font/input[1]').click()



In [364]:
# obtendo a estrutura html da pagina e transformando em um objeto 'soup'
html = browser.page_source
soup = bs.BeautifulSoup(html, 'html.parser')

In [365]:
# Colunas 

# td[0] = num_registro
# td[1] = dta_prioridade
# td[2] = marca
# td[3] = status
# td[4] = titular
# td[5] = ncl


In [366]:
mtx_itens = []
itens = []

In [367]:
tb = soup.find_all('table')[2].find('tbody').find_all('tr')

In [368]:
# para saber quantos marcas a consulta retornou, basta verificar o tamanho da variavel
# que guarda todas as linhas e contar, subtraindo o header e footer
len(tb) - 2

1

In [369]:
tb

[<tr>
 <td align="center" bgcolor="#B5D6AD" height="10" width="5%"><font class="normal"><b>Número</b></font></td>
 <td align="center" bgcolor="#B5D6AD" height="10" width="5%"><font class="normal"><b>Prioridade</b></font></td>
 <td align="center" bgcolor="#B5D6AD" height="18" width="5%"><font class="normal"> </font></td>
 <td align="center" bgcolor="#B5D6AD" height="10" width="25%"><font class="normal"><b>Marca</b></font></td>
 <td align="center" bgcolor="#B5D6AD" height="18" width="5%"><font class="normal"> </font></td>
 <td align="center" bgcolor="#B5D6AD" height="10" width="5%"> <font class="normal"><b>Situação</b></font></td>
 <td align="center" bgcolor="#B5D6AD" height="10" width="30%"><font class="normal"><b>Titular</b></font></td>
 <td align="center" bgcolor="#B5D6AD" height="10" width="20%"><font class="normal"><b>Classe</b></font></td>
 </tr>,
 <tr bgcolor="#E0E0E0" class="normal">
 <td align="center">
 <font class="normal">
 <a class="visitado" href="/pePI/servlet/MarcasServle

MATRIZ DE MARCAS 


In [370]:
range(len(tb)-1)

range(0, 2)

In [371]:
for i in (range(len(tb) - 1)):
    itens = []
    print(i)
    for j in (range(len(tb[i].find_all('td')))):
        
        itens.append( tb[i].find_all('td')[j].get_text().strip() )
    
    mtx_itens.append(itens)
    



0
1


In [372]:
header = mtx_itens[0]  # Primeira linha da matriz
data = mtx_itens[1:]  # Resto dos dados da matriz

In [373]:
df_marcas = pd.DataFrame(data=data, columns=header)

In [374]:
df_marcas

,Número,Prioridade,,Marca,,Situação,Titular,Classe
0,930686195,06/06/2023,,XPRO NUTRITION,,Aguardando manifestação sobre oposição,CR NUTRITION SUPLEMENTOS EIRELI,NCL(12) 05


ESPECIFICACOES DAS MARCAS

In [375]:
for k in range(0, (len(tb) - 2)):
    print(k)
    browser.find_element(By.XPATH,'//*[@id="principal"]/table[3]/tbody/tr[{}]/td[1]/font/a'.format(str((k+2)))).click()
    
    
    
    

0


In [376]:
html = browser.page_source
soup = bs.BeautifulSoup(html, 'html.parser')

 Indices de informacoes 

//*[@id="principal"]/div[5] = classificacao_produto_servico

//*[@id="principal"]/div[10] = classificacao_viena

//*[@id="principal"]/div[13] = titulares

//*[@id="principal"]/div[18] = representante_legal

//*[@id="principal"]/div[21] = datas

//*[@id="principal"]/div[26] = peticoes

//*[@id="principal"]/div[0000] = publicacoes


In [377]:


specs = soup.find(id='principal')

ESPECIFICACOES DAS MARCAS: CLASSIFICACAO

In [378]:
# specs.find_all('div')[i]
# obetendo os nomes de cada coluna
lst_classification = []
range_1 = specs.find_all('div')[5].find_all('table')[0].find_all('th')

for i in (range(len(range_1))):
   lst_classification.append( specs.find_all('div')[5].find_all('table')[0].find_all('th')[i].get_text().strip())


In [379]:
# teste de manipulacao de conteudo lista
lst_classification[0] = str(lst_classification[0] + ': {}'.format(specs.find_all('div')[5].find_all('table')[0].find_all('tr')[1].find_all('td')[0].find('a').get_text().strip()))
lst_classification[1] = str(lst_classification[1] + ': {}'.format(specs.find_all('div')[5].find_all('table')[0].find_all('tr')[1].find_all('td')[5].get_text().strip()))
lst_classification[2] = str(lst_classification[2] + ': {}'.format(specs.find_all('div')[5].find_all('table')[0].find_all('tr')[1].find_all('td')[6].find('a').get_text().strip()))

In [380]:
lst_classification[1]

'Situação da Classe: Vide Situação do Processo'

In [381]:

classificacao = ' \n '.join(lst_classification)
    

In [382]:
mtx_itens[0].append('Classificacao')

In [383]:
mtx_itens[1].append(classificacao)

In [384]:
mtx_itens

[['Número',
  'Prioridade',
  '',
  'Marca',
  '',
  'Situação',
  'Titular',
  'Classe',
  'Classificacao'],
 ['930686195',
  '06/06/2023',
  '',
  'XPRO NUTRITION',
  '',
  'Aguardando manifestação sobre oposição',
  'CR NUTRITION SUPLEMENTOS EIRELI',
  'NCL(12) 05',
  'Classe de Nice: NCL(12) 05 \n Situação da Classe: Vide Situação do Processo \n Especificação: Complemento/suplemento alimentar composto por cereais [não m...']]

Classificacao Viena 


In [385]:
specs.find_all('div')[10].find_all('th')

[<th width="15%"><font class="normal">Edição</font></th>,
 <th width="180"><font class="normal">Código</font></th>,
 <th width="600"><font class="normal">Descrição</font></th>]

In [386]:
header_viena = []
itens = []
for i in range(len(specs.find_all('div')[10].find_all('th'))):
    itens.append(specs.find_all('div')[10].find_all('th')[i].get_text().strip())


header_viena.append(itens)
itens = []


In [387]:
header_viena

[['Edição', 'Código', 'Descrição']]

In [388]:
range(len(specs.find_all('div')[10].find_all('tbody')[0].find_all('tr')[0].find_all('td')))

range(0, 3)

In [389]:
itens = []

for i in range(len(specs.find_all('div')[10].find_all('tbody')[0].find_all('tr'))):
    
    itens = []
    print('p'+ str(i))
    
    for j in range(len(specs.find_all('div')[10].find_all('tbody')[0].find_all('tr')[0].find_all('td'))):
        
        print('j'+str(j))
        itens.append(specs.find_all('div')[10].find_all('tbody')[0].find_all('tr')[i].find_all('td')[j].get_text().strip())
        
    header_viena.append(itens)
itens = []

p0
j0
j1
j2
p1
j0
j1
j2


In [390]:
header_viena

[['Edição', 'Código', 'Descrição'],
 ['4', '27.5.23', 'Várias letras'],
 ['4', '27.5.1', 'Letras apresentando um grafismo especial']]

In [391]:

mtx_itens[0].append('Classificacao Viena')
mtx_itens[1].append(header_viena)

In [392]:
mtx_itens

[['Número',
  'Prioridade',
  '',
  'Marca',
  '',
  'Situação',
  'Titular',
  'Classe',
  'Classificacao',
  'Classificacao Viena'],
 ['930686195',
  '06/06/2023',
  '',
  'XPRO NUTRITION',
  '',
  'Aguardando manifestação sobre oposição',
  'CR NUTRITION SUPLEMENTOS EIRELI',
  'NCL(12) 05',
  'Classe de Nice: NCL(12) 05 \n Situação da Classe: Vide Situação do Processo \n Especificação: Complemento/suplemento alimentar composto por cereais [não m...',
  [['Edição', 'Código', 'Descrição'],
   ['4', '27.5.23', 'Várias letras'],
   ['4', '27.5.1', 'Letras apresentando um grafismo especial']]]]

Titulares

In [393]:
specs.find_all('div')[13].find_all('table')[0].find_all('tr')[0].find_all('th')



[<th width="20"><font class="normal"></font></th>,
 <th width="400"><font class="normal">Nome</font></th>]

In [394]:
header_titular = []
itens = []
for i in range(len(specs.find_all('div')[13].find_all('table')[0].find_all('tr')[0].find_all('th'))):
    
    if (len(specs.find_all('div')[13].find_all('table')[0].find_all('tr')[0].find_all('th')[i].get_text().strip()) == 0):
        itens.append('Ordem')
    else:
        itens.append(specs.find_all('div')[13].find_all('table')[0].find_all('tr')[0].find_all('th')[i].get_text().strip())
header_titular.append(itens)

In [395]:
header_titular

[['Ordem', 'Nome']]

In [396]:
specs.find_all('div')[13].find_all('table')[0].find_all('tr')[1].find_all('td')

[<td width="20"><font class="normal">Titular(1):</font></td>,
 <td width="400"><font class="normal">CR NUTRITION SUPLEMENTOS EIRELI
                              </font></td>]

In [397]:
itens = []
for i in range(len(specs.find_all('div')[13].find_all('table')[0].find_all('tr') )):
    
    
    for j in range(len(specs.find_all('div')[13].find_all('table')[0].find_all('tr')[i].find_all('td'))):

        itens.append(specs.find_all('div')[13].find_all('table')[0].find_all('tr')[i].find_all('td')[j].get_text().strip())
        
    header_titular.append(itens) 
    itens = []   

    
    
    
    


In [398]:
header_titular.pop(1)

[]

In [399]:
header_titular

[['Ordem', 'Nome'], ['Titular(1):', 'CR NUTRITION SUPLEMENTOS EIRELI']]

In [400]:
mtx_itens[0].append('Titulares')
mtx_itens[1].append(header_titular)


In [401]:
mtx_itens

[['Número',
  'Prioridade',
  '',
  'Marca',
  '',
  'Situação',
  'Titular',
  'Classe',
  'Classificacao',
  'Classificacao Viena',
  'Titulares'],
 ['930686195',
  '06/06/2023',
  '',
  'XPRO NUTRITION',
  '',
  'Aguardando manifestação sobre oposição',
  'CR NUTRITION SUPLEMENTOS EIRELI',
  'NCL(12) 05',
  'Classe de Nice: NCL(12) 05 \n Situação da Classe: Vide Situação do Processo \n Especificação: Complemento/suplemento alimentar composto por cereais [não m...',
  [['Edição', 'Código', 'Descrição'],
   ['4', '27.5.23', 'Várias letras'],
   ['4', '27.5.1', 'Letras apresentando um grafismo especial']],
  [['Ordem', 'Nome'], ['Titular(1):', 'CR NUTRITION SUPLEMENTOS EIRELI']]]]

Representante Legal

In [402]:
specs.find_all('div')[18].find_all('tr')

[<tr bgcolor="#b5d6ad">
 <th width="20"><font class="normal"></font></th>
 <th width="400"><font class="normal">Nome</font></th>
 </tr>,
 <tr align="center" bgcolor="#e9e9e9">
 <td width="20"><font class="normal">Procurador:</font></td>
 <td width="400"><font class="normal">Roger Fernando Assunção</font></td>
 </tr>]

In [403]:
header_representante = []
itens = []

for i in range(len(specs.find_all('div')[18].find_all('tr')[0].find_all('th'))):
    
    if (len(specs.find_all('div')[18].find_all('tr')[0].find_all('th')[i].get_text().strip()) == 0):
        continue
    else:
        itens.append(specs.find_all('div')[18].find_all('tr')[0].find_all('th')[i].get_text().strip())
header_representante.append(itens)
    

In [404]:
header_representante

[['Nome']]

In [405]:
# specs.find_all('div')[18].find_all('tr')[].find_all('td')

itens = []
for i in range(len(specs.find_all('div')[18].find_all('table')[0].find_all('tr') )):
    
    
    for j in range(len(specs.find_all('div')[18].find_all('table')[0].find_all('tr')[i].find_all('td'))):

        itens.append(specs.find_all('div')[18].find_all('table')[0].find_all('tr')[i].find_all('td')[j].get_text().strip())
        
    header_representante.append(itens) 
    itens = []   

In [406]:
header_representante.pop(1)

[]

In [407]:
header_representante[1].pop(0)

'Procurador:'

In [408]:
header_representante

[['Nome'], ['Roger Fernando Assunção']]

In [409]:
mtx_itens[0].append('Representante Legal')
mtx_itens[1].append(header_representante)

In [410]:
mtx_itens

[['Número',
  'Prioridade',
  '',
  'Marca',
  '',
  'Situação',
  'Titular',
  'Classe',
  'Classificacao',
  'Classificacao Viena',
  'Titulares',
  'Representante Legal'],
 ['930686195',
  '06/06/2023',
  '',
  'XPRO NUTRITION',
  '',
  'Aguardando manifestação sobre oposição',
  'CR NUTRITION SUPLEMENTOS EIRELI',
  'NCL(12) 05',
  'Classe de Nice: NCL(12) 05 \n Situação da Classe: Vide Situação do Processo \n Especificação: Complemento/suplemento alimentar composto por cereais [não m...',
  [['Edição', 'Código', 'Descrição'],
   ['4', '27.5.23', 'Várias letras'],
   ['4', '27.5.1', 'Letras apresentando um grafismo especial']],
  [['Ordem', 'Nome'], ['Titular(1):', 'CR NUTRITION SUPLEMENTOS EIRELI']],
  [['Nome'], ['Roger Fernando Assunção']]]]

Datas

In [435]:
specs.find_all('div')[21].find_all('table')[0].find_all('tr')[0]

<tr bgcolor="#b5d6ad">
<th width="260"><font class="normal">Data de Depósito</font></th>
<th width="260"><font class="normal">Data de Concessão</font></th>
<th width="260"><font class="normal">Data de Vigência</font>
<font class="normal">
<a class="normal" href="javascript:void(0)" onclick="DisableHide()" onmouseout="hideMe('helpdatavigencia')" onmouseover="showMe('helpdatavigencia')">
<img border="0" height="15" name="class" src="../jsp/imagens/help.gif" width="13"/>
</a>
</font>
<div id="helpdatavigencia" style="BACKGROUND-COLOR: #ffffff; BORDER-BOTTOM: #000000 1px; BORDER-LEFT: #000000 1px; BORDER-RIGHT: #000000 1px; BORDER-TOP: #000000 1px; HEIGHT: 20px; POSITION: absolute; VISIBILITY: hidden; WIDTH: 300px; Z-INDEX: 10; layer-background-color: #FFFFFF">
<table border="1" bordercolor="#006363" cellpadding="5" cellspacing="0" width="100%">
<tbody><tr>
<td align="center" bgcolor="#b5d6ad">
<table border="0" cellpadding="0" cellspacing="0" width="100%">
<tbody><tr>
<td><font class="nor

In [458]:
header_datas = []
itens=[]

for i in range(len(specs.find_all('div')[21].find_all('table')[0].find_all('tr')[0].find_all('th'))):
    
    itens.append(specs.find_all('div')[21].find_all('table')[0].find_all('tr')[0].find_all('th')[i].find('font').get_text().strip())
header_datas.append(itens)

In [459]:
header_datas

[['Data de Depósito', 'Data de Concessão', 'Data de Vigência']]

In [460]:
itens = []
for i in range(len(specs.find_all('div')[21].find_all('table')[0].find_all('tr'))-4):
    
    for j in range(len(specs.find_all('div')[21].find_all('table')[0].find_all('tr')[(i + 4)].find_all('th'))):
        itens.append(specs.find_all('div')[21].find_all('table')[0].find_all('tr')[(i + 4)].find_all('th')[j].get_text().strip())
header_datas.append(itens)

In [462]:
header_datas

[['Data de Depósito', 'Data de Concessão', 'Data de Vigência'],
 ['06/06/2023', '', '']]

Peticoes 

In [471]:
specs.find_all('div')[26].find_all('table')[0].find_all('th')

[<th width="10"><font class="normal">Pgo</font>
 </th>,
 <th width="50"><font class="normal">Protocolo</font>
 </th>,
 <th width="62"><font class="normal">Data</font>
 </th>,
 <th width="10"><font class="normal">Img</font>
 </th>,
 <th width="10">
 </th>,
 <th width="30"><font class="normal">Serviço</font>
 </th>,
 <th width="246"><font class="normal">Cliente</font>
 </th>,
 <th width="10"><font class="normal">Delivery</font>
 </th>,
 <th width="10"><font class="normal">Data</font>
 </th>]

In [480]:
header_peticoes = []
itens = []

for i in range(len(specs.find_all('div')[26].find_all('table')[0].find_all('th'))-1):
    itens.append(specs.find_all('div')[26].find_all('table')[0].find_all('th')[i].get_text().strip())
    
header_peticoes.append(itens)

itens = []

In [481]:
header_peticoes

[['Pgo', 'Protocolo', 'Data', 'Img', '', 'Serviço', 'Cliente', 'Delivery']]

In [498]:
specs.find_all('div')[26].find_all('table')[0].find_all('tr')[1].find_all('td')[0]

<td align="center" bgcolor="#b5d6ad">
<table border="0" cellpadding="0" cellspacing="0" width="100%">
<tbody><tr>
<td><font class="normal">Informações do Banco:</font></td>
<td width="30">
<a href="javascript:EnableHide('pgto0');"><img border="0" height="16" name="class" src="../jsp/imagens/bt_layer.gif" width="26"/></a>
</td>
</tr>
</tbody></table>
</td>